In [1]:
import json
import os
import pandas as pd
import glob



In [48]:
def all_csv(indir_name, df_list):
    for i, f in enumerate(glob.glob(os.path.join(f"{indir_name}","**", "*entries*.csv"), recursive=True)):
        head, tail = os.path.split(f)
        print(i, tail)
        df = pd.read_csv(f)
        df["timestamp"] = df["0"]
        df[["date","time"]] = df["1"].str.split("T", 1, expand=True)
        df["sgv"] = df["2"]
        df_mod = df[["date","sgv"]]
        df2 = df_mod.groupby("date", as_index=False).agg(["mean", "std", "min", "max", "count"])
        # df2.reset_index()
        columns = []
        for col in df2.columns:
            columns.append(f"{col[0]}_{col[1]}")
        df2.columns = columns
        df2.fillna(value=0, inplace=True)  #
        df2.reset_index(inplace=True)
        df2["filename"] = tail
        df_list.append(df2)
        


In [49]:
indir = "/home/reinhold/Daten/OPEN/OPENonOH_Data/csv"
outfile_name = "entries_OPENonOH.csv"
df_list = []
all_csv(indir, df_list)
df = pd.concat(df_list, axis=0)
#print(df)
#df.info()
df.to_csv(os.path.join(indir, outfile_name))
print(os.path.join(indir, outfile_name) + " created")


0 00977838_23987999_entries_2020-09-01_to_2020-11-08.csv
1 00977838_23989488_entries_2020-09-01_to_2020-11-08.csv
2 00749582_69754_entries__to_2018-06-07.csv
3 00749582_999974_entries__to_2019-02-15.csv
/home/reinhold/Daten/OPEN/OPENonOH_Data/csv/entries_OPENonOH.csv created
